In [ ]:
import itertools
import ipywidgets as widgets

import numpy as np
from ipycanvas import Canvas, MultiCanvas

import bokeh
from bokeh.io import output_notebook, show, push_notebook

output_notebook(bokeh.resources.INLINE)

from bokeh.models import ColumnDataSource, Circle, Line
import bokeh.plotting

Some parameters that can be tweaked (carefully, though)

In [ ]:
width, height = 900, 600 # in pixels
# width, height = 100, 600 # in pixels

# these pixels are encompassed by those above
horizontal_padding = 300 # in pixels
vertical_padding = 100

# the smaller, the easier the sensors get activated
obs_to_angle_sensitivity = 0.25

tx_power = 2e3

sensors_fill_color = 'red'
position_fill_color = 'green'

n_sensors = 4

# sensors are represented as circles
sensors_radius = 19

# Model

We compute the observation recorded at a sensor placed at position $\mathbf{s}$ when the *target* is at $\mathbf{x}$  as
$$
    y
    =
    \frac{
        P_0
    }
    {
        \left|\left|
            \mathbf{x} - \mathbf{s}
        \right|\right|_2
    }
$$
where $\left|\left|\mathbf{x}\right|\right|_2$ is the modulus (2-norm) of vector $\mathbf{x}$.

In [ ]:
def obs(x_1: int, y_1: int, x_sensor: int, y_sensor) -> float:
    """
        Computes an observation given the x and y coordinates for two different positions.
    """
    
    return tx_power / np.sqrt((x_1 - x_sensor)**2 + (y_1 - y_sensor)**2)

The functions going from the model observation to the angle in the *canvas*' meters is
$$
2\pi
\left(
\frac{
    1
}{
    1 + e^{-\rho y}
}
- 0.5
\right)
/
(1-0.5)
$$
where $\rho$ is the above *observation-to-angle sensitivity* (`obs_to_angle_sensitivity`).

In [ ]:
def obs_to_angle(obs: float) -> float:
    """
        Turns a measure into an angle in radians.
    """
    
    return 2*np.pi * ((1. / (1. + np.exp(-obs_to_angle_sensitivity*obs))) - 0.5) / (1 - 0.5)

# Preliminary computations

In [ ]:
# number of sensors per axis
n_sensors_per_axis = int(np.sqrt(n_sensors))

# all the x coordinates of the sensors...
sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis)
# sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis+)[1:]

# ...and all the y's
sensors_y_coordinates = np.linspace(vertical_padding, height-vertical_padding, n_sensors_per_axis)

# a list of tuples with the resulting positions
sensors_coordinates = list(itertools.product(sensors_x_coordinates, sensors_y_coordinates))

n_sensors = len(sensors_coordinates)

*Global* variables are used to keep track of the clicked positions.

In [ ]:
target_position = []
observations = []

# Canvas

We instantiate a 2-layers canvas: one for the sensors (fixed), and one for the (user-recorded) positions.

In [ ]:
canvas = MultiCanvas(n_canvases=2, width=width, height=height)

## Background

The filling style and color for the background canvas ($\#0$) are set up.

In [ ]:
canvas[0].fill_style = sensors_fill_color
canvas[0].stroke_style = 'blue'

# Border

The region in which the target can move.

In [ ]:
canvas[0].stroke_rect(0, 0, canvas[0].width, canvas[0].height)

# Sensors

Sensors are plotted and, for each one of them we build a `plotting.figure` with its corresponding `ColumnDataSource`. Additionally, an `widgets.Output` is also instantiated for every figure/sensor. This is done only once.

In [ ]:
output_plots = []
figures, sources = [], []

# for every sensor's x-y coordinates
for x, y in sensors_coordinates:
    
    # the sensor is plotted (outlined)
    canvas[0].stroke_arc(x, y, 20, 0, 2*np.pi)
    
    figures.append(bokeh.plotting.figure(plot_width=400, plot_height=300, y_range=(0, 40), toolbar_location=None))
    sources.append(ColumnDataSource(data=dict(x=[0], y=[0])))
    line = Line(x="x", y="y")
    figures[-1].add_glyph(sources[-1], line)
    
    # a new output widget is created for the corresponding plot
    output_plots.append(widgets.Output())

A grid layout (using horizontal and vertical *containers*) is set to hold the above individual *widgets*.

In [ ]:
columns = []

for i in range(0, n_sensors, n_sensors_per_axis):
    
    columns.append(widgets.VBox(output_plots[i:i+n_sensors_per_axis]))

grid = widgets.HBox(columns)

## Events handlers

Handler for when mouse is *un-clicked*

In [ ]:
def handle_mouse_up(x, y):
    
    # the clicked position is "recorded"
    target_position.append((x,y))
    
    # a squared dot representing the clicked position (color must be set every time; see below)
    canvas[1].fill_style = position_fill_color
    canvas[1].fill_rect(x, y, 10, 10)
    
    # observations associated with the new position
    new_observations = [None] * n_sensors
    
    # for every sensor (x and y coordinates)...
    for i_sensor, (x_sensor, y_sensor) in enumerate(sensors_coordinates):
        
        # the previous filling in the corresponding "intensity meter" is erased
        canvas[1].fill_style = 'white'
        canvas[1].fill_arc(x_sensor, y_sensor, sensors_radius, 0, 2*np.pi)
        
        # the new observation is computed
        new_observations[i_sensor] = obs(x, y, x_sensor, y_sensor)
        
        # the intensity meter is adjusted accordingly
        canvas[1].fill_style = sensors_fill_color
        canvas[1].fill_arc(x_sensor, y_sensor, sensors_radius, 0, obs_to_angle(new_observations[i_sensor]))
    
    # the new observations are appended to the collection (with the previous ones)
    observations.append(new_observations)
    
    # for the sake of convenience
    observations_np = np.array(observations)
    
    x = np.arange(observations_np.shape[0])

    # for every plot's source and handler (associated with a sensor) along with the newly added observations...
    for source, handler, y in zip(sources, handlers, observations_np.T):
        
        # ...data points are updated...
        source.data = {'x':x, 'y':y}
        
        # ...and a "redraw" enforced
        push_notebook(handle=handler)

canvas[1].on_mouse_up(handle_mouse_up)

---

The grid is shown and the and the individual widgets it encompasses are populated with bokeh figures.

In [ ]:
display(grid)

handlers = []

for figure, output_plot in zip(figures, output_plots):
    
    with output_plot:
        
        handlers.append(show(figure, notebook_handle=True))

Next cell provides the canvas. By clicking any point on the *map* we record the position of the target at a (discrete) time instant. This has two effects:
- Each sensor fills up to the extent of its received signal strength (starting from $0$ degrees and going clock-wise).
- The plots above are updated with the signal seen so far by every sensor (you need to click more than once for this to be show).

It is probably a good idea to **zoom out** a little bit to see the canvas and the plots simultaneously.

In [ ]:
canvas